In [15]:
import pandas as pd
import numpy as np
from glob import glob


In [16]:

file_lst = glob("raw/*.csv")


In [17]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [22]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    

    print(len(r))    
    
    rng = np.random.default_rng()
    index=np.sort(rng.choice([i for i in range(size)], size=int(size*0.1), replace=False, shuffle=False))
    r.add(df.to_numpy()[index])
    del df
    


7
0
100000
200000
300000
301440
388920
488920
491028
491393
512260
579513
679513
740686
741257
741539
742288
842288
865786
927073
1027073
1027698
1028190


In [23]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 9540336


In [24]:
df = r.getDf()
df.to_csv("processed/imbalanced_dataset.csv", index=False)

In [25]:


df = pd.read_csv("processed/imbalanced_dataset.csv")
print(df[df.isnull().any(axis=1)].shape[0])
N=df.shape[0]
train_size=N
print(f"N: {N}, TS:{train_size}")
test_df = df.loc[train_size:N]
df = df.loc[0:train_size]

0
N: 1029537, TS:1029537


In [26]:
test_df.to_csv('processed/test_data.csv', index=False)

In [27]:
# from imblearn.over_sampling import ADASYN
# Y = df['label'].to_numpy().astype(np.uint8)
# X = df.drop('label', axis=1).to_numpy()
# df[df['label'].isnull()].head()
# print(Y[0:100])



In [28]:
# from imblearn.over_sampling import ADASYN
# X_r, Y_r = ADASYN().fit_resample(X, Y)
# res=np.empty((X_r.shape[0], df.shape[1]))
# res[:, 0:-1] = X_r
# res[:, -1] = Y_r
# pd.DataFrame(res, columns = df.columns).to_csv("processed/balanced_dataset.csv", index=False)

In [29]:
# np.bincount(Y_r)

In [30]:
# np.random.shuffle(res)


In [31]:
df.to_csv("processed/train_data.csv", index=False)

In [32]:
df.tail(15)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,dst_pkts,label
1029522,0.000308,41.0,89.0,0.0,1.0,1.0,0.0
1029523,0.000663,0.0,178.0,0.0,0.0,2.0,0.0
1029524,0.000419,41.0,89.0,0.0,1.0,1.0,0.0
1029525,0.000522,0.0,212.0,0.0,0.0,2.0,0.0
1029526,0.000402,31.0,106.0,0.0,1.0,1.0,0.0
1029527,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
1029528,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
1029529,0.000455,0.0,178.0,0.0,0.0,2.0,0.0
1029530,0.000573,0.0,212.0,0.0,0.0,2.0,0.0
1029531,0.000450,0.0,178.0,0.0,0.0,2.0,0.0
